# PyCoMo Maximum Growth-Rate #
This tutorial show-cases how to find the maximum growth-rate of a community - across all possible abundance profiles.

The expected runtime for this notebook is less than 10 minutes.

In [1]:
from pathlib import Path
import sys
import os
import cobra
import math 
import time
import warnings

In [2]:
import pycomo
pycomo.configure_logger(level="info")

2025-11-24 16:01:29,406 - PyCoMo - INFO - Logger initialized.


## Load the Toy Models ##
These are simple toy models of 2 organisms. In a community, they are expexted to grow at a maximum of 10/h, however one organism alone can grow as fast as 15/h. Both cases can be calculated.

In [3]:
# create two cobra models from sbml (change the path according to where the models are located)
toy1 = cobra.io.read_sbml_model("../data/use_case/toy_models/toy_1.xml")
toy2 = cobra.io.read_sbml_model("../data/use_case/toy_models/toy_2_2.xml")

# create Single Organism Models from cobra models
Toy1 = pycomo.SingleOrganismModel(toy1, "toy1")
Toy2 = pycomo.SingleOrganismModel(toy2, "toy2")

# create Community Model from Single Organism Models
C = pycomo.CommunityModel([Toy1, Toy2], name = "Toy_community")
# instantiate the model and set cplex as a solver (if available)
#C.model.solver = "cplex"

## Compute maximum community growth rate

The method ```max_growth_rate``` calculates the overall maximum growth rate of the community, regardless of its composition.

In [4]:
C.max_growth_rate()

2025-11-24 16:01:29,498 - PyCoMo - INFO - No community model generated yet. Generating now:
2025-11-24 16:01:29,505 - PyCoMo - INFO - Identified biomass reaction from objective: R4
2025-11-24 16:01:29,522 - PyCoMo - INFO - Identified biomass reaction from objective: R4
2025-11-24 16:01:29,553 - PyCoMo - WARNING - Not all reactions in the model are mass and charge balanced. To check which reactions are imbalanced, please run the get_unbalanced_reactions method of this CommunityModel object
2025-11-24 16:01:29,553 - PyCoMo - INFO - Generated community model.
2025-11-24 16:01:29,554 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-11-24 16:01:31,855 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:31,890 - PyCoMo - INFO - New round: lb: 10.0, ub: 1000.0, x: 10.000002
2025-11-24 16:01:34,232 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:34,264 - PyCoMo - INFO - New round: lb: 15.000000000000002, ub: 1000.0, x: 15.000002000000002
2025-11-24 

np.float64(15.0)

```max_growth_rate``` has the following parameters:
* ```minimal_abundance``` (default = 0)
* ```return_abundances``` (default = False)
* ```sensitivity``` (default = 6)
* ```gurobi``` (default = False)

### Return Abundances
By setting the parameter ```return_abundances``` to ```True```, additional information about the feasible community compositions at the maximum growth rate is returned in the form of a pandas dataframe.

The minimal flux of x_fraction_reaction corresponds to the minimal abundance of member x in the community. Likewise, the maximal flux corresponds to the maximal abundance of member x.

In [5]:
C.max_growth_rate(return_abundances=True)

2025-11-24 16:01:38,975 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-11-24 16:01:41,724 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:41,762 - PyCoMo - INFO - New round: lb: 10.0, ub: 1000.0, x: 10.000002
2025-11-24 16:01:44,298 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:44,332 - PyCoMo - INFO - New round: lb: 15.000000000000002, ub: 1000.0, x: 15.000002000000002
2025-11-24 16:01:46,793 - PyCoMo - WARNING - worker 22840: toy1_fraction_reaction min flux is infeasible
2025-11-24 16:01:46,794 - PyCoMo - WARNING - worker 22840: toy1_fraction_reaction max flux is infeasible
2025-11-24 16:01:46,795 - PyCoMo - WARNING - worker 22840: toy2_fraction_reaction min flux is infeasible
2025-11-24 16:01:46,796 - PyCoMo - WARNING - worker 22840: toy2_fraction_reaction max flux is infeasible
2025-11-24 16:01:46,797 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:46,826 - PyCoMo - WARNING - Not all upper bounds are bigger than 

,reaction_id,min_flux,max_flux
0,toy1_fraction_reaction,0.0,0.0
1,toy2_fraction_reaction,1.0,1.0
2,community_biomass,15.0,15.0


### Minimal abundance
Edge cases exist in which the highest growth rate of a community is not achieved by a combination of all the members. Instead, one member with an abundance of 100% may achieve the maximum growth rate. 

In order to exclude these cases, the parameter ```minimal_abundance``` must be set to a float greater than 0. This sets the minimal abundance of all members to the set float. Take care that the sum of the minimal abundances is not greater than 1, as this will result in an error.

In [6]:
C.max_growth_rate(minimal_abundance=0.1)

2025-11-24 16:01:51,646 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-11-24 16:01:54,179 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:54,210 - PyCoMo - INFO - New round: lb: 10.0, ub: 1000.0, x: 10.000002
2025-11-24 16:01:56,748 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:56,778 - PyCoMo - WARNING - Not all upper bounds are bigger than the set minimal abundance:
toy1_fraction_reaction    0.0
toy2_fraction_reaction    1.0
Name: max_flux, dtype: float64
2025-11-24 16:01:56,779 - PyCoMo - INFO - New round: lb: 10.0, ub: 10.000002, x: 10.000001000000001
2025-11-24 16:01:59,343 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:01:59,373 - PyCoMo - WARNING - Not all upper bounds are bigger than the set minimal abundance:
toy1_fraction_reaction    0.0
toy2_fraction_reaction    1.0
Name: max_flux, dtype: float64
2025-11-24 16:01:59,373 - PyCoMo - INFO - New round: lb: 10.0, ub: 10.000001000000001, x: 10.0000005
2025-11-24 1

np.float64(10.0)

In [7]:
# too large minimal_abundance value leads to an error
try:
    C.max_growth_rate(minimal_abundance=0.6)
except ValueError:
    print("Error thrown!")
    
# The community has two members: 
# 2 * 0.6 = 1.2
# 1.2 > 1

Error thrown!


## Example: Biogas producing community ##
The example for this part will be a three member community published by Koch et al. 2019 (https://doi.org/10.1371/journal.pcbi.1006759). The three member organisms are representatives of functional guilds in a biogas community.

In [8]:
# create cobra models from sbml
dv = cobra.io.read_sbml_model("../data/use_case/koch/dv.xml")
mh = cobra.io.read_sbml_model("../data/use_case/koch/mh.xml")
mb = cobra.io.read_sbml_model("../data/use_case/koch/mb.xml")

# change infinite upper bounds to 1000
for model in [dv,mh,mb]:
    for reaction in model.reactions:
        if reaction.upper_bound == math.inf:
            reaction.upper_bound = 1000.
        if reaction.lower_bound == -math.inf:
            reaction.lower_bound = -1000.

# create Single Organism Models from cobra models
DV = pycomo.SingleOrganismModel(dv, "dv")
MH = pycomo.SingleOrganismModel(mh, "mh")
MB = pycomo.SingleOrganismModel(mb, "mb")

# create Community Model from Single Organism Models
C2 = pycomo.CommunityModel([DV, MH, MB], name = "dv_mh_mb_community")

# set solver to cplex (if available)
#C2.model.solver = "cplex"

# apply the same medium as in the paper by Koch et al.
medium = {
    'EX_CO2_EX_medium': 1000.0,
    'EX_Eth_EX_medium': 1000.0,
    'EX_BM_tot_medium': 1000.0
}
C2.medium = medium
C2.apply_medium()

# Formate and Hydrogen are not allowed to accumulate in the medium.
C2.model.reactions.get_by_id("EX_Form_EX_medium").upper_bound = 0.
C2.model.reactions.get_by_id("EX_H2_EX_medium").upper_bound = 0.


'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
'2PG__PEP' is not a valid SBML 'SId'.
'3PG__2PG' is not a valid SBML 'SId'.
'0Pyr__AcCoA' is not a valid SBML 'SId'.
'5CHOMPT' is not a valid SBML 'SId'.
'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
'2PG__3PG' is not a valid SBML 'SId'.
'3PG__DPG' is not a valid SBML 'SId'.
'5CHOMPT__CHH4MPT' is not a valid SBML 'SId'.
'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
'5CHOMPT' is not a valid SBML 'SId'.
'3PG__2PG__3PG' is not a valid SBML 'SId'.
'5CHOMPT__CHH4MPT' is not a valid SBML 'SId'.
2025-11-24 16:02:02,176 - PyCoMo - INFO - No community model generated yet. Generating now:
2025-11-24 16:02:02,195 - PyCoMo - INFO - Identified biomass reaction from objective: r_BMDV2BMc
2025-11-24 16:02:02,344 - PyCoMo - INFO - Identified biomass reaction from objective: BM_Synth
2025-11-24 16:02:02,562 - PyCoMo - INFO - Identified biomass reaction from objective: BM_Synth
2025-11-24 16:02:02,7

In [9]:
# calculate max growth rate of community
C2.max_growth_rate()

2025-11-24 16:02:02,895 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-11-24 16:02:05,505 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:02:05,587 - PyCoMo - INFO - New round: lb: 0.052128854257330506, ub: 1000.0, x: 0.05213085425733051
2025-11-24 16:02:05,665 - PyCoMo - INFO - New round: lb: 0.052128854257330506, ub: 0.05213085425733051, x: 0.05212985425733051
2025-11-24 16:02:05,707 - PyCoMo - INFO - New round: lb: 0.052128854257330506, ub: 0.05212985425733051, x: 0.05212935425733051
2025-11-24 16:02:08,307 - PyCoMo - WARNING - worker 19340: dv_fraction_reaction min flux is infeasible
2025-11-24 16:02:08,308 - PyCoMo - WARNING - worker 34356: mh_fraction_reaction min flux is infeasible
2025-11-24 16:02:08,309 - PyCoMo - WARNING - worker 19340: dv_fraction_reaction max flux is infeasible
2025-11-24 16:02:08,311 - PyCoMo - WARNING - worker 34356: mh_fraction_reaction max flux is infeasible
2025-11-24 16:02:08,336 - PyCoMo - INFO - Processed 100.0% of fva

np.float64(0.052128)

In [10]:
# calculate abundance profile as well
C2.max_growth_rate(return_abundances=True)

2025-11-24 16:02:08,414 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-11-24 16:02:11,056 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:02:11,133 - PyCoMo - INFO - New round: lb: 0.052128854257330506, ub: 1000.0, x: 0.05213085425733051
2025-11-24 16:02:11,210 - PyCoMo - INFO - New round: lb: 0.052128854257330506, ub: 0.05213085425733051, x: 0.05212985425733051
2025-11-24 16:02:11,252 - PyCoMo - INFO - New round: lb: 0.052128854257330506, ub: 0.05212985425733051, x: 0.05212935425733051
2025-11-24 16:02:13,837 - PyCoMo - WARNING - worker 20308: dv_fraction_reaction min flux is infeasible
2025-11-24 16:02:13,838 - PyCoMo - WARNING - worker 29620: mh_fraction_reaction min flux is infeasible
2025-11-24 16:02:13,839 - PyCoMo - WARNING - worker 29620: mh_fraction_reaction max flux is infeasible
2025-11-24 16:02:13,840 - PyCoMo - WARNING - worker 20308: dv_fraction_reaction max flux is infeasible
2025-11-24 16:02:13,865 - PyCoMo - INFO - Processed 100.0% of fva

,reaction_id,min_flux,max_flux
0,dv_fraction_reaction,0.074553,0.393590
1,mh_fraction_reaction,0.000000,0.681848
2,mb_fraction_reaction,0.000000,0.925447
3,community_biomass,0.052128,0.052128


### Sensitivity

The ```sensitivity``` parameter describes the amount of decimal places that should be calculated. The default is set to 6, as the cplex solver may return inaccurate results at a higher value.

In [11]:
# sensitivity = 4
C2.max_growth_rate(sensitivity=4)

2025-11-24 16:02:16,548 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 0.0002
2025-11-24 16:02:19,495 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:02:19,593 - PyCoMo - INFO - New round: lb: 0.052128854257330624, ub: 1000.0, x: 0.05232885425733062
2025-11-24 16:02:19,684 - PyCoMo - INFO - New round: lb: 0.052128854257330624, ub: 0.05232885425733062, x: 0.05222885425733062
2025-11-24 16:02:19,769 - PyCoMo - INFO - Maximum growth-rate is 0.0521


np.float64(0.0521)

In [12]:
# sensitivity = 2
C2.max_growth_rate(sensitivity=2)

2025-11-24 16:02:19,777 - PyCoMo - INFO - New round: lb: 0.0, ub: 1000.0, x: 0.02
2025-11-24 16:02:22,398 - PyCoMo - INFO - Processed 100.0% of fva steps
2025-11-24 16:02:22,471 - PyCoMo - INFO - New round: lb: 0.05212885425733061, ub: 1000.0, x: 0.07212885425733061
2025-11-24 16:02:22,552 - PyCoMo - INFO - New round: lb: 0.05212885425733061, ub: 0.07212885425733061, x: 0.06212885425733061
2025-11-24 16:02:22,593 - PyCoMo - INFO - New round: lb: 0.05212885425733061, ub: 0.06212885425733061, x: 0.05712885425733061
2025-11-24 16:02:22,669 - PyCoMo - INFO - Maximum growth-rate is 0.05


np.float64(0.05)